## 1. 引入

在上一章，我们介绍了因子分析，其给出了一种将 $n$ 维数据 $x$ 在 $k$ 维子空间中建模的方法。其基于概率模型，通过 EM 算法来估计参数。

本章我们将介绍另一种降维方法：**「主成分分析(Principal Component Analysis)」法**（PCA）。该方法更加直接，只需要特征向量的计算，不需要 EM 求解。

给定一个数据集 $\{x^{(i)};i=1,...,m\}$，其中 $x^{(i)}\in \mathbb R^n(n \ll m)$。假定该数据集来自对无线电控制直升机飞行员的调查，而其中的两个属性分别为飞行员的技能评估和其对飞行的感兴趣长度。

考虑到飞行的特殊性，这两个属性是存在 **「正相关」** 关系的，即实际上数据的信息量是 $n-1$ 维的。PCA 解决的就是如何将多余属性去除的问题。

将上述两个属性使用坐标图进行展示，得到：


可以看到，$u_1$ 展示出了数据之间的相关性，称之为“主方向”；$u_2$ 则代表主方向之外的噪声。PCA 要做的就是找到 **「主方向」**，并将数据投影到该方向，达到降维的目的。

## 2. 算法流程

### 2.1 预处理

在运行 PCA 算法之前，需要首先进行预处理来归一化数据均值及方差：

1. 令 $\mu=\frac{1}{m}\sum_{i=1}^m x^{(i)}$

2. 使用 $x^{(i)}-\mu$ 来替代 $x^{(i)}$

3. 令 $\sigma_j^2 = \frac{1}{m}\sum_{i}(x_j^{(i)})^2$

4. 使用 $\frac{x_j^{(i)}}{\sigma_j}$ 来替代 $x^{(i)}$

前两步将数据的均值变为 0（已知均值为 0 时可以省略）；后两步将数据每个维度的方差变为 1（已知数据各维度处于同一尺度下时可以省略）。

### 2.2 计算主方向

计算主方向的方法之一是找到单位向量 $u$，使得数据在该方向上的投影的 **「方差最大化」** 。

直观上来看，原始数据必然存在一定的方差（信息），而我们希望投影后的数据（降维后）在子空间尽量保留原始数据的信息，即方差最大化。

以如下数据集为例（归一化已完成）：

可以看到，图 2 方向上的投影相比图 3 距离原点更远，方差更大。我们希望能够自动找到类似图 2 的方向。

下面给出寻找该方向的方法：给定一个单位向量 $u$ 和一个点 $x$，其投影长度（距离原点）为 $x^T u$

因此，为了最大化投影的方差，即最大化：

$$\frac{1}{m}\sum_{i=1}^m({x^{(i)}}^T u)^2=\frac{1}{m}\sum_{i=1}^m u^T x^{(i)}{x^{(i)}}^Tu=u^T\Big(\frac{1}{m}\sum_{i=1}^m x^{(i)}{x^{(i)}}^T\Big)u$$

对于归一化后的数据，其投影点的均值也为 0，因此方差计算为直接平方。该公式有一个约束条件：$\parallel u \parallel_2=1$。

利用拉格朗日方程，可以求得该最大化问题的解即为经验协方差矩阵 $\Sigma=\frac{1}{m}\sum_{i=1}^m x^{(i)}{x^{(i)}}^T$ 的 **「主要特征向量」**。构建如下拉格朗日方程：

$$\ell = u^T\Sigma u=\lambda(u^T u-1)$$